In [1]:
import torch
import numpy as np 
import pandas as pd

/dssg/home/acct-clsyzs/clsyzs/.conda/envs/esm_dds/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [43]:
x = torch.load("./x_train_multi.pt")

/tmp/ipykernel_1473388/3399517390.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x = torch.load("./x_train_multi.pt")
/tmp/ipykernel_1473388/3399517390.py:2: FutureWarn

In [11]:
x_test = torch.load("./x_test_multi.pt")

In [2]:
x_test_sim_rmsd = torch.load("x_test_sim_rmsd.pt")

In [3]:
x_train_sim_rmsd = torch.load("x_train_sim_rmsd.pt")

# Filter the dataset based on different conditions.

In [4]:
def dict_sim_rmsd(dict_1, sim = None,rmsd = None):
    dict_2 = {}
    for i in dict_1:
        dict_3 = {}
        for j in dict_1[i]:
            if sim != None and rmsd!= None:
                if dict_1[i][j][-1][1]<=sim and dict_1[i][j][-1][0]>=rmsd:
                    dict_3[j] = dict_1[i][j]#[:2]
            elif sim != None and rmsd== None:
                if dict_1[i][j][-1][1]<=sim:
                    dict_3[j] = dict_1[i][j]#[:2]
            elif sim == None and rmsd != None:
                if dict_1[i][j][-1][0]>=rmsd:
                    dict_3[j] = dict_1[i][j]#[:2]
        if dict_3 != {}:
            dict_2[i] = dict_3
    return dict_2
                    

In [15]:
dict_2 = dict_sim_rmsd(x_test_sim_rmsd,0.2,0)
# Filter the dataset according to sequence similarity and RMSD values.

In [27]:
ns = 0
for i in dict_2:
    ns+=len(dict_2[i])
print("Identify dual-target pairs that satisfy the specified conditions:", ns)

Identify dual-target pairs that satisfy the specified conditions: 3076


In [28]:
import pandas as pd

data_all = pd.read_csv('./pdb_2021aug02/list.csv')
cluster = data_all.CLUSTER.tolist()
chainid = data_all.CHAINID.tolist()
lst_chain_cluster = {}
for i in range(len(chainid)):
    lst_chain_cluster[chainid[i]] = cluster[i]

In [29]:
lst_name = []
lst_cluster_s = []
dict_chainid_cluster = {}
for i in dict_2:
    for j in dict_2[i]:
        
        if lst_chain_cluster[x_test[j]["name"]] in dict_chainid_cluster.keys():
            if list(set([i,j])) not in dict_chainid_cluster[lst_chain_cluster[x_test[j]["name"]]]:
                dict_chainid_cluster[lst_chain_cluster[x_test[j]["name"]]].append(list(set([i,j])))
                
        else:
            dict_chainid_cluster[lst_chain_cluster[x_test[j]["name"]]] = [list(set([i,j]))]
        

In [33]:
ns = 0
for i in dict_chainid_cluster:
    ns+=len(dict_chainid_cluster[i])
print("Identify dual-target clusters that satisfy the specified conditions:", len(dict_chainid_cluster)) 

In [35]:
for i in dict_chainid_cluster:
    print("cluster_id:",i,"total pair",len(dict_chainid_cluster[i]))

cluster_id: 24229 total pair 1
cluster_id: 17266 total pair 1106
cluster_id: 18108 total pair 227
cluster_id: 782 total pair 86
cluster_id: 25072 total pair 36
cluster_id: 4860 total pair 22
cluster_id: 26075 total pair 638
cluster_id: 21329 total pair 36
cluster_id: 26367 total pair 7
cluster_id: 289 total pair 3
cluster_id: 14424 total pair 1
cluster_id: 20936 total pair 13
cluster_id: 14914 total pair 3
cluster_id: 16080 total pair 4
cluster_id: 6441 total pair 69
cluster_id: 14163 total pair 3
cluster_id: 9620 total pair 15
cluster_id: 27679 total pair 68
cluster_id: 950 total pair 267
cluster_id: 17633 total pair 22
cluster_id: 2767 total pair 1
cluster_id: 3552 total pair 11
cluster_id: 17437 total pair 5
cluster_id: 9964 total pair 1
cluster_id: 20865 total pair 1
cluster_id: 18932 total pair 4
cluster_id: 23505 total pair 6
cluster_id: 10354 total pair 3
cluster_id: 961 total pair 2
cluster_id: 27104 total pair 4
cluster_id: 2546 total pair 3
cluster_id: 1207 total pair 3
clust

# Use a stratified sampling strategy.

In [42]:
#sim 0.3
len_cluster = {}
ns = 0
for i in dict_chainid_cluster:
    len_cluster[i] = len(dict_chainid_cluster[i])
    if len(dict_chainid_cluster[i])>100:
        ns+=10
    if 10<=len(dict_chainid_cluster[i])<=100:
        ns+=2
    else:
        ns+=1

In [36]:
#rmsd 2
len_cluster = {}
ns = 0
for i in dict_chainid_cluster:
    len_cluster[i] = len(dict_chainid_cluster[i])
    if len(dict_chainid_cluster[i])>100:
        ns+=10
    if 10<=len(dict_chainid_cluster[i])<=100:
        ns+=3
    else:
        ns+=1

In [39]:
import random
lst_dict_x = {}
for i in dict_chainid_cluster:
    lst_pair = dict_chainid_cluster[i]
    
    len1 = len(lst_pair)
    
    if len1<=10:
         
        idx = random.randint(0, len1-1)
        keys1,keys2 = lst_pair[idx]
        if keys1 in x_test_sim_rmsd.keys():
            if keys2 in x_test_sim_rmsd[keys1].keys():
                lst_dict_x[keys1] = {keys2: x_test_sim_rmsd[keys1][keys2]}
            else:
                lst_dict_x[keys2] = {keys1: x_test_sim_rmsd[keys2][keys1]}
        
        else:
            if keys1 in x_test_sim_rmsd[keys2].keys():
                lst_dict_x[keys2] = {keys1: x_test_sim_rmsd[keys2][keys1]}
            else:
                lst_dict_x[keys1] = {keys2: x_test_sim_rmsd[keys1][keys2]}
    elif 10<len1<=100:
         
        idx_lst = random.sample(list(range(len1)), 3)
        
        for idx in idx_lst:
            keys1,keys2 = lst_pair[idx]
            if keys1 in x_test_sim_rmsd.keys():
                if keys2 in x_test_sim_rmsd[keys1].keys():
                    lst_dict_x[keys1] = {keys2: x_test_sim_rmsd[keys1][keys2]}
                else:
                    lst_dict_x[keys2] = {keys1: x_test_sim_rmsd[keys2][keys1]}
        
            else:
                if keys1 in x_test_sim_rmsd[keys2].keys():
                    lst_dict_x[keys2] = {keys1: x_test_sim_rmsd[keys2][keys1]}
                else:
                    lst_dict_x[keys1] = {keys2: x_test_sim_rmsd[keys1][keys2]}
    else:
         
        idx_lst = random.sample(list(range(len1)), 10)
        
        for idx in idx_lst:
            keys1,keys2 = lst_pair[idx]
            if keys1 in x_test_sim_rmsd.keys():
                if keys2 in x_test_sim_rmsd[keys1].keys():
                    lst_dict_x[keys1] = {keys2: x_test_sim_rmsd[keys1][keys2]}
                else:
                    lst_dict_x[keys2] = {keys1: x_test_sim_rmsd[keys2][keys1]}
        
            else:
                if keys1 in x_test_sim_rmsd[keys2].keys():
                    lst_dict_x[keys2] = {keys1: x_test_sim_rmsd[keys2][keys1]}
                else:
                    lst_dict_x[keys1] = {keys2: x_test_sim_rmsd[keys1][keys2]}

In [15]:
lst_dict_x = dict_x_test 

In [41]:
for i in lst_dict_x:
    for j in lst_dict_x[i]:
        print(x_test[i]["name"],x_test[j]["name"],lst_chain_cluster[x_test[i]["name"]],lst_dict_x[i],i)

2bcx_A 1cm4_A 17266 {11155: [['A'], ['A'], [6.585686544675358, 0.2222222222222222]]} 9805
6b8n_B 4byf_B 17266 {21465: [['B'], ['B'], [7.584153092331256, 0.09497964721845319]]} 2696
6k4r_C 6paw_C 17266 {16004: [['A'], ['A'], [12.225381775151913, 0.19242273180458624]]} 13098
6xxf_A 6k4l_C 17266 {22453: [['A'], ['A'], [12.049541576695795, 0.024054982817869417]]} 21246
3bxl_A 1k90_B 17266 {14190: [['A'], ['B'], [13.348811336655693, 0.03913894324853229]]} 762
2x0g_B 2f3y_A 17266 {24491: [['B'], ['A'], [3.462889652528007, 0.05970149253731343]]} 5528
6ynu_A 1sk6_B 17266 {20905: [['A'], ['B'], [8.774017129855627, 0.04296875]]} 17475
1sk6_B 2f3y_A 17266 {24491: [['B'], ['A'], [13.49808264607841, 0.041176470588235294]]} 20905
6b8l_B 6sz5_A 17266 {13321: [['B'], ['C'], [14.025339529679249, 0.0942622950819672]]} 4554
4byf_B 4jpz_C 17266 {24697: [['B'], ['A'], [4.759281827736864, 0.15491183879093198]]} 21465
4mgi_B 6uzk_B 25072 {12285: [['B'], ['B'], [3.314856303062204, 0.20665083135391923]]} 2839


In [100]:
torch.save(lst_dict_x,"dict_x_test_30_159.pt")